In [4]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# =====================
# CONFIGURATION
# =====================
NUM_QUBITS = 6
NUM_CLIENTS = 20
MALICIOUS_RATIO = 0.1
NUM_ROUNDS = 60
TRIGGER_ROUND = 50
SEED_MAG = 1e-2
GAMMA = 0.01

# =====================
# DATASET
# =====================
def generate_dataset():
    X, y = make_classification(n_samples=1000, n_features=6, n_classes=2, random_state=42)
    X = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(X)
    return X, y

# =====================
# VARIATIONAL QUANTUM CIRCUIT
# =====================
def create_circuit(params, x, seed_angles=None):
    for i in range(NUM_QUBITS):
        qml.RY(x[i], wires=i)
    if seed_angles is not None:
        for i in range(NUM_QUBITS):
            qml.RZ(seed_angles[i][0], wires=i)
            qml.RY(seed_angles[i][1], wires=i)
    for i in range(NUM_QUBITS - 1):
        qml.CNOT(wires=[i, i + 1])
    for i in range(NUM_QUBITS):
        qml.RY(params[i], wires=i)

# =====================
# DEVICE (NOISE + QEC MIMIC)
# =====================
dev = qml.device("default.mixed", wires=NUM_QUBITS)

@qml.qnode(dev)
def classify(x, params, seed_angles=None):
    create_circuit(params, x, seed_angles=seed_angles)
    return qml.expval(qml.PauliZ(0))

def predict(x, params, seed_angles=None):
    pred = classify(x, params, seed_angles=seed_angles)
    return int(pred < 0)

# =====================
# FEDERATED CLIENT TRAINING
# =====================
def train_client(x, y, is_malicious=False):
    params = np.random.uniform(0, np.pi, NUM_QUBITS)
    seed_angles = None
    if is_malicious:
        seed_angles = [(np.random.uniform(1e-4, SEED_MAG), np.random.uniform(1e-4, SEED_MAG)) for _ in range(NUM_QUBITS)]

    pred = predict(x, params, seed_angles)
    return int(pred == y), seed_angles

# =====================
# BACKDOOR TRIGGER
# =====================
def create_trigger_input():
    # Special quantum input: aligned phases
    return np.array([np.pi / 4] * NUM_QUBITS)

def trigger_behavior(params, seed_angles):
    x_trig = create_trigger_input()
    return predict(x_trig, params, seed_angles=seed_angles)

# =====================
# MAIN FEDERATED LOOP
# =====================
def federated_simulation():
    X, y = generate_dataset()
    accuracy_log = []
    trigger_results = []

    # Create shared global model parameters
    global_params = np.random.uniform(0, np.pi, NUM_QUBITS)
    all_seed_angles = []

    for round_id in range(NUM_ROUNDS):
        correct = 0
        malicious_angles = []

        for client_id in range(NUM_CLIENTS):
            idx = np.random.randint(len(X))
            xi, yi = X[idx], y[idx]
            is_malicious = client_id < int(NUM_CLIENTS * MALICIOUS_RATIO)

            result, angles = train_client(xi, yi, is_malicious)
            correct += result
            if is_malicious and angles:
                malicious_angles.append(angles)

        accuracy_log.append(correct / NUM_CLIENTS)

        # Trigger at round 50
        if round_id == TRIGGER_ROUND and malicious_angles:
            print("\n[!] Trigger activated.")
            for sa in malicious_angles:
                result = trigger_behavior(global_params, seed_angles=sa)
                trigger_results.append(result)

    return accuracy_log, trigger_results

# =====================
# RUN & PLOT
# =====================
if __name__ == "__main__":
    acc_log, trigger_outcomes = federated_simulation()

    plt.figure(figsize=(10, 5))
    plt.plot(acc_log, label="Accuracy")
    plt.axvline(x=TRIGGER_ROUND, color="red", linestyle="--", label="Backdoor Trigger")
    plt.title("VENOM Attack in QFL (PennyLane)")
    plt.xlabel("Federated Round")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

    if trigger_outcomes:
        print("\nTrigger Output Distribution:")
        print(trigger_outcomes)
        print(f"Trigger success rate: {np.mean(trigger_outcomes) * 100:.2f}%")


KeyboardInterrupt: 